# Combine netCDF files from SARAH3

In [ ]:
from glob import glob
import xarray
from dask.distributed import Client

In [10]:
client = Client(n_workers=12)

/scratch/snx3000/kschuurm/lightning/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35145 instead
  warnings.warn(


In [ ]:
seviri = xarray.open_zarr(
    "/scratch/snx3000/kschuurm/ZARR/SEVIRI_FULLDISK.zarr"
).drop_duplicates(dim="time")
seviri = seviri.rename_dims({"x": "lon", "y": "lat"}).rename_vars({"x": "lon", "y": "lat"})
seviri = seviri.drop_duplicates(dim="time")
seviri

In [ ]:
lat = seviri.lat.copy()
lon = seviri.lon.copy()
# del seviri



def clean_reindex_sarah3(df):
    df = df.sel(time=df.record_status == 0).drop_vars(
        ["record_status", "lat_bnds", "lon_bnds"]
    )
    # df = clean(df)
    return df.reindex(lat=lat, lon=lon, method="nearest")



In [ ]:
fns = glob("/scratch/snx3000/kschuurm/DATA/SARAH3/SIS/SISin*.nc")

sis = xarray.open_mfdataset(
    fns,
    parallel=True,
    engine="h5netcdf",
    chunks={"time": 48, "lat": -1, "lon": -1},
)

In [ ]:
sis = clean_reindex_sarah3(sis).chunk({"time": 48, "lat": -1, "lon": -1})

In [ ]:
a = sis.SIS.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr", mode="a")
del a, sis

In [ ]:
fns = glob("/scratch/snx3000/kschuurm/DATA/SARAH3/CAL/CALin*.nc")

cal = xarray.open_mfdataset(
    fns,
    parallel=True,
    engine="h5netcdf",
    chunks={"time": 48, "lat": -1, "lon": -1},
)
cal

In [ ]:
cal = clean_reindex_sarah3(cal).chunk({"time": 48, "lat": -1, "lon": -1})

In [ ]:
a = cal.CAL.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr", mode="a")
del a, cal

# Rechunk SARAH3 with a single dataarray: channel_data

In [ ]:
sarah = xarray.open_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr")

sarah = sarah.to_dataarray('channel').to_dataset(name="channel_data")

sarah['channel_data'] = sarah.channel_data.chunk({"channel":-1, "time": 48, "lat": -1, "lon": -1})

sarah.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3_new.zarr", mode="a")